In [19]:
"""
The system trains BERT (or any other transformer model like RoBERTa, DistilBERT etc.) on the SNLI + MultiNLI (AllNLI) dataset
with MultipleNegativesRankingLoss. Entailments are positive pairs and the contradiction on AllNLI dataset is added as a hard negative.
At every 10% training steps, the model is evaluated on the STS benchmark dataset

Usage:
python training_nli_v2.py

OR
python training_nli_v2.py pretrained_transformer_model_name
"""

import logging
import sys
import traceback
from datetime import datetime

from datasets import load_dataset

from sentence_transformers import SentenceTransformer, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.similarity_functions import SimilarityFunction
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import BatchSamplers, SentenceTransformerTrainingArguments

# Set the log level to INFO to get more information
logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO)


In [20]:

# model_name = sys.argv[1] if len(sys.argv) > 1 else "distilroberta-base"
model_name = 'distilroberta-base'
train_batch_size = 128  # The larger you select this, the better the results (usually). But it requires more GPU memory
max_seq_length = 75
num_epochs = 1
print("Model name is: ", model_name)
# Save path of the model
output_dir = (
    "output/training_nli_v2_" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)


Model name is:  distilroberta-base


In [21]:
# 1. Here we define our SentenceTransformer model. If not already a Sentence Transformer model, it will automatically
# create one with "mean" pooling.
model = SentenceTransformer(model_name)


2024-08-08 20:42:30 - Use pytorch device_name: mps
2024-08-08 20:42:30 - Load pretrained SentenceTransformer: distilroberta-base
2024-08-08 20:42:31 - No sentence-transformers model found with name distilroberta-base. Creating a new one with mean pooling.


In [22]:
# 2. Load the AllNLI dataset: https://huggingface.co/datasets/sentence-transformers/all-nli
# We'll start with 10k training samples, but you can increase this to get a stronger model
logging.info("Read AllNLI train dataset")
train_dataset = load_dataset("sentence-transformers/all-nli", "triplet", split="train").select(range(10000))
eval_dataset = load_dataset("sentence-transformers/all-nli", "triplet", split="dev").select(range(1000))
logging.info(train_dataset)

2024-08-08 20:42:32 - Read AllNLI train dataset
2024-08-08 20:42:41 - Dataset({
    features: ['anchor', 'positive', 'negative'],
    num_rows: 10000
})


In [23]:
print(train_dataset[0:1])

{'anchor': ['A person on a horse jumps over a broken down airplane.'], 'positive': ['A person is outdoors, on a horse.'], 'negative': ['A person is at a diner, ordering an omelette.']}


In [24]:

# 3. Define our training loss: https://sbert.net/docs/package_reference/sentence_transformer/losses.html#multiplenegativesrankingloss
train_loss = losses.MultipleNegativesRankingLoss(model)



In [25]:

# 4. Define an evaluator for use during training. This is useful to keep track of alongside the evaluation loss.
stsb_eval_dataset = load_dataset("sentence-transformers/stsb", split="validation")
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=stsb_eval_dataset["sentence1"],
    sentences2=stsb_eval_dataset["sentence2"],
    scores=stsb_eval_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-dev",
)
logging.info("Evaluation before training:")
dev_evaluator(model)


2024-08-08 20:42:45 - Evaluation before training:
2024-08-08 20:42:45 - EmbeddingSimilarityEvaluator: Evaluating the model on the sts-dev dataset:
2024-08-08 20:42:49 - Cosine-Similarity :	Pearson: 0.6060	Spearman: 0.6375
2024-08-08 20:42:49 - Manhattan-Distance:	Pearson: 0.6696	Spearman: 0.6730
2024-08-08 20:42:49 - Euclidean-Distance:	Pearson: 0.6273	Spearman: 0.6338
2024-08-08 20:42:49 - Dot-Product-Similarity:	Pearson: 0.1756	Spearman: 0.1868


{'sts-dev_pearson_cosine': 0.6060232703288173,
 'sts-dev_spearman_cosine': 0.6374793753666167,
 'sts-dev_pearson_manhattan': 0.669590412518936,
 'sts-dev_spearman_manhattan': 0.672964509781593,
 'sts-dev_pearson_euclidean': 0.6273032479364506,
 'sts-dev_spearman_euclidean': 0.6337971367146357,
 'sts-dev_pearson_dot': 0.17563152785707975,
 'sts-dev_spearman_dot': 0.18683610958401164,
 'sts-dev_pearson_max': 0.669590412518936,
 'sts-dev_spearman_max': 0.672964509781593}

In [26]:

# 5. Define the training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=output_dir,
    # Optional training parameters:
    num_train_epochs=1,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=train_batch_size,
    warmup_ratio=0.1,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=10,
    save_strategy="steps",
    save_steps=10,
    save_total_limit=2,
    logging_steps=100,
    run_name="nli-v2",  # Will be used in W&B if `wandb` is installed
)


In [27]:
# import torch
# device = torch.device('mps')


# 6. Create the trainer & start training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=train_loss,
    evaluator=dev_evaluator,
)
trainer.train()


ValueError: fp16 mixed precision requires a GPU (not 'mps').

In [ ]:

# 7. Evaluate the model performance on the STS Benchmark test dataset
test_dataset = load_dataset("sentence-transformers/stsb", split="test")
test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=test_dataset["sentence1"],
    sentences2=test_dataset["sentence2"],
    scores=test_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-test",
)
test_evaluator(model)


2024-08-08 20:42:23 - EmbeddingSimilarityEvaluator: Evaluating the model on the sts-test dataset:
2024-08-08 20:42:27 - Cosine-Similarity :	Pearson: 0.5289	Spearman: 0.5463
2024-08-08 20:42:27 - Manhattan-Distance:	Pearson: 0.6027	Spearman: 0.5937
2024-08-08 20:42:27 - Euclidean-Distance:	Pearson: 0.5439	Spearman: 0.5432
2024-08-08 20:42:27 - Dot-Product-Similarity:	Pearson: 0.1353	Spearman: 0.1365


{'sts-test_pearson_cosine': 0.5289057255119349,
 'sts-test_spearman_cosine': 0.5463460134341998,
 'sts-test_pearson_manhattan': 0.6027049108434683,
 'sts-test_spearman_manhattan': 0.593713509693665,
 'sts-test_pearson_euclidean': 0.5438941838259634,
 'sts-test_spearman_euclidean': 0.5431951670662212,
 'sts-test_pearson_dot': 0.13531686469301124,
 'sts-test_spearman_dot': 0.13649247202993078,
 'sts-test_pearson_max': 0.6027049108434683,
 'sts-test_spearman_max': 0.593713509693665}

In [ ]:

# 8. Save the trained & evaluated model locally
final_output_dir = f"{output_dir}/final"
model.save(final_output_dir)
